In [1]:
import pandas as pd
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests 
from bs4 import BeautifulSoup
from env_miatta import github_token, github_username
import time


# ACQUIRE

In [2]:
# Define your URL and headers
# url = 'https://github.com/'
# headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}
# response = requests.get(url, headers=headers)


In [3]:
REPOS = ['learn-co-curriculum/phase-0-the-dom-editing-lab',
 'yankouskia/additional_5',
 'linuxacademy/cicd-pipeline-train-schedule-dockerdeploy',
 'brentley/ecsdemo-nodejs']
 # 'CorentinJ/Real-Time-Voice-Cloning',
 # 'ripienaar/free-for-dev',
 # 'hasura-imad/imad-2016-app',
 # 'supabase/supabase',
 # 'scutan90/DeepLearning-500-questions',
 # 'travis-ci/docs-travis-ci-com',
 # '233boy/v2ray',
 # 'thingsboard/thingsboard',
 # 'Azure/azure-quickstart-templates',
 # 'XX-net/XX-Net',
 # 'typicode/json-server',
 # 'bloominstituteoftechnology/github-usercard',
 # 'learn-co-curriculum/phase-1-javascript-variables-labJS-Challenges/recursion-prompts',
 # 'AtsushiSakai/PythonRobotics',
 # 'kodekloudhub/certified-kubernetes-administrator-course',
 # 'jackfrued/Python-100-Days',
 # 'spring-projects/spring-framework',
 # 'lin-xin/vue-manage-system',
 # 'forem/forem',
 # 'deadlyvipers/dojo_rules',
 # 'learn-co-students/javascript-objects-bootcamp-prep--000',
 # 'bloominstituteoftechnology/User-Interface',
 # 'RasaHQ/rasa',
 # 'ColorlibHQ/gentelella',
 # 'facebookresearch/fastText',
 # 'raulmur/ORB_SLAM2',
 # 'tastejs/todomvc',
 # 'vuejs/v2.vuejs.org',
 # 'barryclark/jekyll-now',
 # 'ceph/ceph',
 # 'bloominstituteoftechnology/React-Todo',
 # 'espressif/esp-idf',
 # 'Significant-Gravitas/Auto-GPT',
 # 'celery/celery',
 # 'learn-co-curriculum/phase-0-html-issue-bot-9000-lab',
 # 'MakeSchool/FlappyBirdTemplate-Spritebuilder',
 # 'learn-co-students/python-dictionaries-readme-data-science-intro-000',
 # 'apache/kafka',
 # 'jeecgboot/jeecg-boot',
 # 'JetBrains/kotlin',
 # 'micropython/micropython',
 # 'learn-co-students/js-from-dom-to-node-bootcamp-prep-000',
 # 'vaxilu/x-ui',
 # 'learn-co-students/python-variables-lab-data-science-intro-000',
 # 'TheOdinProject/javascript-exercises',
 # 'rapid7/metasploit-framework',
 # 'zhisheng17/flink-learning',
 # 'GitbookIo/gitbook',
 # 'ant-design/ant-design-pro',
 # 'mitmproxy/mitmproxy',
 # 'learn-co-students/js-beatles-loops-lab-bootcamp-prep-000',
 # 'progedu/adding-up',
 # 'altercation/solarized',
 # 'learn-co-curriculum/react-hooks-lists-and-keys-lab',
 # 'TheAlgorithms/Python',
 # 'ultralytics/yolov5',
 # 'dcxy/learngit',
 # 'cyclic-software/starter-express-api',
 # 'bloominstituteoftechnology/Preprocessing-II',
 # 'heartcombo/devise',
 # 'learn-co-students/python-lists-lab-data-science-intro-000',
 # 'learn-co-students/js-deli-counter-bootcamp-prep-000',
 # 'bloominstituteoftechnology/DOM-II',
 # 'torvalds/linux',
 # 'Binaryify/NeteaseCloudMusicApi',
 # 'namndwebdev/tang-crush',
 # 'jquery/jquery',
 # 'luchihoratiu/debug-via-ssh',
 # 'keycloak/keycloak',
 # 'PowerShellMafia/PowerSploit',
 # 'bilibili/ijkplayer',
 # 'Ebazhanov/linkedin-skill-assessments-quizzes',
 # 'learn-co-students/js-if-else-files-lab-bootcamp-prep-000learn-co-students/first-ide-lab-bootcamp-prep-000',
 # 'josephmisiti/awesome-machine-learning',
 # 'jtleek/datasharing',
 # 'Turonk/infra_actions',
 # 'bloominstituteoftechnology/module-challenge-intro-to-git',
 # 'moby/moby',
 # 'jenkinsci/jenkins',
 # 'modood/Administrative-divisions-of-China',
 # 'linuxacademy/cicd-pipeline-train-schedule-gradle',
 # 'learn-co-students/jupyter-notebook-introduction-data-science-intro-000',
 # 'taizilongxu/interview_python',
 # 'bloominstituteoftechnology/node-db1-project',
 # 'lazyprogrammer/machine_learning_examples',
 # 'stacksimplify/azure-aks-kubernetes-masterclass',
 # 'trekhleb/homemade-machine-learning',
 # 'freefq/free',
 # 'xitu/gold-miner',
 # 'kubernetes/kubernetes',
 # 'xingshaocheng/architect-awesome',
 # 'mqyqingfeng/Blog',
 # 'forezp/SpringCloudLearning',
 # 'qmk/qmk_firmware',
 # 'learn-co-students/javascript-intro-to-functions-lab-bootcamp-prep-000',
 # 'apache/flink',
 # 'angular/angular-phonecat',
 # 'RedHatTRaining/DO288-apps',
 # 'alx-tools/your_first_code',
 # 'codefresh-contrib/gitops-certification-examples',
 # 'reduxjs/redux',
 # 'learn-co-students/python-dictionaries"-readme-data-science-intro-000',
 # 'amjuarez/bytecoin',
 # 'devopshydclub/vprofile-project',
 # 'docsifyjs/docsify',
 # 'learn-co-students/javascript-arrays-lab-bootcamp-prep-000',
 # 'trustwallet/assets',
 # 'vivienzou1/DL-Notes-for-interview',
 # 'learn-co-students/javascript-logging-lab-bootcamp-prep-000',
 # 'othneildrew/Best-README-Template',
 # 'jumpserver/jumpserver',
 # 'bloominstituteoftechnology/team-builder',
 # '996icu/996.ICU',
 # 'developerforce/intro-to-heroku',
 # 'selectize/selectize.js',
 # 'protocolbuffers/protobuf',
 # 'ibm-developer-skills-network/xzceb-flask_eng_fr',
 # 'nightscout/cgm-remote-monitor']
 
url = 'https://github.com/'
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_contents = requests.get(get_readme_download_url(contents)).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)

In [4]:
words_df = scrape_github_data()


In [5]:
# Aquire data using methods described above.
words_df = pd.read_json('data2.json')
words_df.head()



,repo,language,readme_contents
0,learn-co-curriculum/phase-0-the-dom-editing-lab,JavaScript,# DOM Editing Lab\n\n## Learning Goals\n\n- Id...
1,yankouskia/additional_5,JavaScript,# Brackets\n\n## Task\n\nImplement function `c...
2,linuxacademy/cicd-pipeline-train-schedule-dock...,JavaScript,# cicd-pipeline-train-schedule-dockerdeploy\n\...
3,brentley/ecsdemo-nodejs,Shell,![Build Status](https://codebuild.us-east-2.am...
